In [30]:
!pip install kaggle

     |████████████████████████████████| 58 kB 2.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 149 kB 4.9 MB/s            
     |████████████████████████████████| 62 kB 2.3 MB/s             
     |████████████████████████████████| 76 kB 4.8 MB/s             
     |████████████████████████████████| 138 kB 6.1 MB/s            
     |████████████████████████████████| 78 kB 7.2 MB/s             
     |████████████████████████████████| 61 kB 6.7 MB/s             
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=6734efa97d6e655e88d3f1de257c223238b6893d2c9c7c9d7ea457293f0bb715
  Stored in directory: /home/studio-lab-user/.cache/pip/wheels/ac/b2/c3/fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


In [31]:
!mkdir ~/.kaggle

In [32]:
!pwd

/home/studio-lab-user/sagemaker-studiolab-notebooks/HW1P2


In [34]:
!kaggle competitions download -c idl-fall2021-hw1p2

100%|██████████████████████████████████████▊| 2.39G/2.40G [00:26<00:00, 115MB/s]
100%|██████████████████████████████████████| 2.40G/2.40G [00:27<00:00, 95.5MB/s]


In [1]:
!pwd

/home/studio-lab-user/sagemaker-studiolab-notebooks/HW1P2


In [2]:
!unzip idl-fall2021-hw1p2.zip -d Data/

Archive:  idl-fall2021-hw1p2.zip
  inflating: Data/dev.npy            
Data/dev.npy:  write error (disk full?).  Continue? (y/n/^C) ^C


In [3]:
!y

/usr/bin/sh: 1: y: not found


# Global Settings

In [1]:
%pwd
GLOBALTEST = False

In [2]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import time
import torch.optim as optim

from torch.utils import data

%matplotlib inline

In [3]:
cuda = torch.cuda.is_available()
if cuda: print("Using cuda!!") 
else: print("No cuda available")

Using cuda!!


# Loading Data

In [4]:
train_data = np.load("Data/toy_train_data.npy", allow_pickle=True)
train_label = np.load("Data/toy_train_label.npy", allow_pickle=True)
test_data = np.load("Data/toy_val_data.npy", allow_pickle=True)
test_label = np.load("Data/toy_val_label.npy", allow_pickle=True)

In [5]:
print(train_data.shape)
print(train_data[0].shape)

(1000,)
(1184, 40)


In [6]:
print(train_label.shape)
print(train_label[0].shape)

(1000,)
(1184,)


## Dataset

In [7]:
class MyDataset(data.Dataset):
    def __init__(self, data, label, context_size = 1):
        # data and label nparrays directly loaded from files
        N = len(data)
        X = []
        Y = []
        locator = []
        ctr = 0
        for i in range(N):
            xi = torch.from_numpy(data[i]).float()
            yi = torch.from_numpy(label[i]).long()
            X.append(xi)
            Y.append(yi)
            li = xi.shape[0]
            for j in range(li):
                locator.append((i, j)) #ith piece, jth time frame
                ctr += 1
        
        self.X = X
        self.Y = Y
        self.locator = locator
        self.context_size = context_size

    def __len__(self):
        return len(self.locator)

    def __getitem__(self,index):
        i, j = self.locator[index]
        left = j - self.context_size
        right = j + self.context_size # both sides inclusive
        if GLOBALTEST: print(left, right) 
        X = self.X[i][j:j+1]
        if left < 0:
            XL = self.X[i][:j]
            if GLOBALTEST: print(XL.shape) 
            XL = F.pad(input=XL, pad=(0, 0, self.context_size-XL.shape[0], 0), mode='constant', value=0)
        else:
            XL = self.X[i][left:j]
        
        if right >= self.X[i].shape[0]:
            
            XR = self.X[i][j+1:]
            XR = F.pad(input=XR, pad=(0, 0, 0, self.context_size-XR.shape[0]), mode='constant', value=0)
        else:
            XR = self.X[i][j+1:right+1]
        
        if GLOBALTEST:
            print("XL shape:", XL.shape)
            print("X shape:", X.shape)
            print("XR shape:", XR.shape)
        
        X = torch.cat((XL,X,XR), 0).reshape(-1)
        Y = self.Y[i][j]
        
        return X,Y

## Dataloader

In [8]:
num_workers = 4 if cuda else 0 
context_size = 5

# Training
train_dataset = MyDataset(train_data, train_label, context_size = context_size)

train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=64)
train_loader = data.DataLoader(train_dataset, **train_loader_args)

# Testing
test_dataset = MyDataset(test_data, test_label, context_size = context_size)

test_loader_args = dict(shuffle=False, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
test_loader = data.DataLoader(test_dataset, **test_loader_args)

# Model and Loss Function

In [9]:
class Simple_MLP(nn.Module):
    def __init__(self, size_list):
        super(Simple_MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
            layers.append(nn.Linear(size_list[i],size_list[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [10]:
model = Simple_MLP([40*(2*context_size+1), 256, 128, 64, 71])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

Simple_MLP(
  (net): Sequential(
    (0): Linear(in_features=440, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=71, bias=True)
  )
)


# Training Procedure

In [11]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

In [12]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.to(device)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()


        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Testing Loss: ', running_loss)
        print('Testing Accuracy: ', acc, '%')
        return running_loss, acc

## Training

In [ ]:
n_epochs = 20
save = True
Train_loss = []
Test_loss = []
Test_acc = []

for i in range(n_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    test_loss, test_acc = test_model(model, test_loader, criterion)
    Train_loss.append(train_loss)
    Test_loss.append(test_loss)
    Test_acc.append(test_acc)
    if save:
        torch.save(model.state_dict(), "checkpoint.pth")
    print('='*20)

Training Loss:  1.3542344530683506 Time:  22.649085521697998 s
Testing Loss:  1.7564266423478512
Testing Accuracy:  50.524049985736355 %
Training Loss:  1.2581681161084723 Time:  22.616214990615845 s
Testing Loss:  1.701590466160345
Testing Accuracy:  51.9392998588455 %
Training Loss:  1.1987913167360824 Time:  22.612998247146606 s
Testing Loss:  1.6958526370649654
Testing Accuracy:  52.39129065601647 %
Training Loss:  1.156319332694856 Time:  22.590041399002075 s
Testing Loss:  1.649172333405481
Testing Accuracy:  53.48570115998622 %
Training Loss:  1.1237769034819873 Time:  22.500128984451294 s
Testing Loss:  1.6430865552188096
Testing Accuracy:  53.96696021369532 %
Training Loss:  1.097457647678187 Time:  22.473544597625732 s
Testing Loss:  1.651182627310685
Testing Accuracy:  53.87063430610151 %
Training Loss:  1.0756994876718795 Time:  22.687042474746704 s
Testing Loss:  1.6432378613270855
Testing Accuracy:  54.15775960758307 %
Training Loss:  1.057174422356573 Time:  22.666249513

## Result Visualization

In [ ]:
plt.title('Loss Plots')
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.plot(Train_loss, label='Training Loss')
plt.plot(Test_loss, label = 'Test Loss')
plt.legend()

In [ ]:
plt.title('Test Accuracy')
plt.xlabel('Epoch Number')
plt.ylabel('Accuracy (%)')
plt.plot(Test_acc)